# Tile Conus PCL

In [ ]:
import os
from itertools import product
import rasterio as rio
from rasterio import windows

in_path = "C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\PCL\\"
input_filename = 'pcl_west_wgs.tif'

out_path = "C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\PCLTILES\\"
output_filename = 'pcltile_{}-{}.tif'

widthtile = 5000
heighttile = 5000

def get_tiles(ds, width=widthtile, height=heighttile):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform
                   
with rio.open(os.path.join(in_path, input_filename)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  
    meta = inds.meta.copy()
    for window, transform in get_tiles(inds):
        print(window)
        data = inds.read(window=window)
        if nodata is not None and not (data == nodata).all():
            meta['transform'] = transform
            meta['width'], meta['height'] = window.width, window.height
            outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
            with rio.open(outpath, 'w', **meta) as outds:
                outds.write(data)


# Download data

In [ ]:
### for DEM
import rioxarray
from pystac_client import Client
import planetary_computer
from concurrent.futures import ThreadPoolExecutor
import rioxarray
import matplotlib.pyplot as plt
import os
import glob
import subprocess
from osgeo import gdal, ogr, osr

# Path to the GeoTIFF file
tif_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\PCLTILES\pcltile_75000-35000.tif"

# Directory for TIFF files
tif_dir = r'C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\scratch\dem'

# Read the GeoTIFF file using rioxarray
tif_data = rioxarray.open_rasterio(tif_path)

# Get the spatial extent of the GeoTIFF
min_lon, min_lat, max_lon, max_lat = tif_data.rio.bounds()
bbox_of_interest = [min_lon, min_lat, max_lon, max_lat]  # Replace with your actual bounding box coordinates

# Create a catalog search for the specified collection and bounding box
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search = catalog.search(collections=["cop-dem-glo-30"], bbox=bbox_of_interest)
items = list(search.get_items())
print(f"Returned {len(items)} items")

def process_item(item, idx):
    #print(f"Processing item {idx+1}/{len(items)}")
    signed_asset = planetary_computer.sign(item.assets["data"])
    data = (
        rioxarray.open_rasterio(signed_asset.href)
        .squeeze()
        .drop("band")
    )
    data.rio.write_crs("EPSG:4326", inplace=True)

    output_tif_path = os.path.join(tif_dir, f"output_dataDEM_{idx}.tif")
    data.rio.to_raster(output_tif_path)

with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers based on your CPU
    for i, item in enumerate(items):  # Change this line to process only the last three items
        executor.submit(process_item, item, len(items) - 3 + i)  # Adjust index for correct labeling

output_tif = os.path.join(tif_dir, f"outputtile_DEM.tif")

# Get the list of TIFF files
tifs = glob.glob(os.path.join(tif_dir, "*.tif"))

# Prepare the gdal_merge command
merge_command_hag = [
    "python",
    "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
    "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
    "-o", output_tif,
    "-n", "-9999",
    "-a_nodata", "-9999"
] + tifs

# Run the gdal_merge command and capture the output
process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)



# print(output_tif)
# print(tif_path)
# output_tif = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\DEMTIF\DEM_CO.tif"
# tif_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\PCLTILES\pcltile_75000-35000.tif"

# Open the source (lidarHAG) and target (pcl) datasets
src_ds = gdal.Open(output_tif, gdal.GA_ReadOnly)
target_ds = gdal.Open(tif_path, gdal.GA_ReadOnly)

# Get the Geotransform and Projection from the target dataset
target_transform = target_ds.GetGeoTransform()
target_projection = target_ds.GetProjection()
target_cols = target_ds.RasterXSize
target_rows = target_ds.RasterYSize

# Create a new dataset for output with the same size and projection as the target
driver = gdal.GetDriverByName('GTiff')
output_path = os.path.join(tif_dir, f"output_resampled_dataDEM.tif")

out_ds = driver.Create(output_path, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
out_ds.SetGeoTransform(target_transform)
out_ds.SetProjection(target_projection)

# Perform the resampling
gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

src_ds = None
target_ds = None
out_ds = None

os.remove(output_tif)

if process_hag.returncode != 0:
    print(process_hag.stderr)
else:
    for tif in tifs:
        try:
            os.remove(tif)
        except Exception as e:
            print(f"Failed to delete {tif}: {e}")

print("Resampling completed. Output saved at:", output_path)

In [ ]:
### for LIDAR
import os
import glob
import subprocess
from osgeo import gdal, ogr, osr

tif_dir = r'C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\scratch\lidar'

tifs = glob.glob(os.path.join(tif_dir, "*.tif"))

output_tif = os.path.join(tif_dir, f"outputtile_lidar.tif")

# Get the list of TIFF files
tifs = glob.glob(os.path.join(tif_dir, "*.tif"))

# Prepare the gdal_merge command
merge_command_hag = [
    "python",
    "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
    "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
    "-o", output_tif,
    "-n", "255",
    "-a_nodata", "255"
] + tifs

# Run the gdal_merge command and capture the output
process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Open the source (lidarHAG) and target (pcl) datasets
src_ds = gdal.Open(output_tif, gdal.GA_ReadOnly)
target_ds = gdal.Open(tif_path, gdal.GA_ReadOnly)

# Get the Geotransform and Projection from the target dataset
target_transform = target_ds.GetGeoTransform()
target_projection = target_ds.GetProjection()
target_cols = target_ds.RasterXSize
target_rows = target_ds.RasterYSize

# Create a new dataset for output with the same size and projection as the target
driver = gdal.GetDriverByName('GTiff')
#output_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\Input\Resampled_DEM_CO.tif"
output_path = os.path.join(tif_dir, f"output_resampled_dataLIDAR.tif")

out_ds = driver.Create(output_path, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
out_ds.SetGeoTransform(target_transform)
out_ds.SetProjection(target_projection)

# Perform the resampling
gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

# Close the datasets
src_ds = None
target_ds = None
out_ds = None

os.remove(output_tif)

print("Resampling completed. Output saved at:", output_path)

In [ ]:
# for Rivers
import rioxarray
import osmnx as ox

# Path to the GeoTIFF file
tif_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\PCLTILES\pcltile_75000-35000.tif"

# Directory for TIFF files
#tif_dir = r'C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\scratch\dem'

# Read the GeoTIFF file using rioxarray
dem_data = rioxarray.open_rasterio(tif_path)

# Get the spatial extent of the GeoTIFF
min_lon, min_lat, max_lon, max_lat = dem_data.rio.bounds()

#custom_filter = '["waterway"~"river|streams|canals"]'
custom_filter = '["waterway"~"river"]'

bbox = (max_lat, min_lat, max_lon, min_lon)  
graph = ox.graph_from_bbox(bbox=bbox, custom_filter=custom_filter, simplify=True, retain_all=True, truncate_by_edge=True)


gdf = ox.graph_to_gdfs(graph, nodes=False)

# Convert the graph to GeoDataFrames
nodes, edges = ox.graph_to_gdfs(graph)

import rasterio
from rasterio.features import rasterize
from rasterio.windows import from_bounds
import numpy as np
import scipy.ndimage

# Define the decay constant
lambda_decay = 0.03  # This value might need adjustment based on your specific use case

# Load the reference TIFF to use for transformations and metadata
with rasterio.open(tif_path) as src:

    min_lon, min_lat, max_lon, max_lat = src.bounds
    
    meta = src.meta.copy()

    # Convert geographic coordinates to pixel coordinates
    window = from_bounds(min_lon, min_lat, max_lon, max_lat, src.transform)
    transform = rasterio.windows.transform(window, src.transform)

    # Correct the window dimensions to integers
    window_width = int(window.width)
    window_height = int(window.height)

    # Create an empty raster array for the size of the window
    raster = np.zeros((window_height, window_width), dtype=np.uint8)

    # Rasterize directly into the window shape
    shapes = ((geom, 1) for geom in gdf['geometry'])
    burned = rasterize(shapes, out=raster, fill=0, transform=transform, all_touched=True)

    # Compute the Euclidean distance from each cell to the nearest non-zero cell
    distance_grid = scipy.ndimage.distance_transform_edt(burned == 0)

    # Apply exponential decay to the distance grid
    decay_grid = np.exp(-lambda_decay * distance_grid)

    # Prepare metadata for saving the decay grid
    clipped_meta = meta.copy()
    clipped_meta.update({
        "driver": "GTiff",
        "height": window_height,
        "width": window_width,
        "transform": transform,
        "dtype": rasterio.float32,
        "count": 1,
        "compress": 'lzw'
    })

    # Save the decay grid to a new TIFF file
    output_path = r'C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\scratch\\rivers\\exponential_decay_CO_river.tif'
    with rasterio.open(output_path, 'w', **clipped_meta) as dst:
        dst.write(decay_grid.astype(np.float32), 1)

from osgeo import gdal, ogr, osr

# Define the paths to your rasters
#lidarHAG_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\exponential_decay_CO_river.tif"
#pcl_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\pcl_west_wgs_CO_3.tif"

# Open the source (lidarHAG) and target (pcl) datasets
src_ds = gdal.Open(output_path, gdal.GA_ReadOnly)
target_ds = gdal.Open(tif_path, gdal.GA_ReadOnly)


# Get the Geotransform and Projection from the target dataset
target_transform = target_ds.GetGeoTransform()
target_projection = target_ds.GetProjection()
target_cols = target_ds.RasterXSize
target_rows = target_ds.RasterYSize

# Create a new dataset for output with the same size and projection as the target
driver = gdal.GetDriverByName('GTiff')
output_path_resample = r"C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\scratch\\rivers\\output_resampled_dataRivers.tif"
out_ds = driver.Create(output_path_resample, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
out_ds.SetGeoTransform(target_transform)
out_ds.SetProjection(target_projection)

# Perform the resampling
gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

# Close the datasets
src_ds = None
target_ds = None
out_ds = None

print("Resampling completed. Output saved at:", output_path)

# Check if the file exists before trying to delete it
if os.path.exists(output_path):
    try:
        os.remove(output_path)
        print(f"File {output_path} deleted successfully.")
    except Exception as e:
        print(f"Failed to delete {output_path}: {e}")
else:
    print(f"No file found at {output_path}, deletion not required.")


In [ ]:
# for Roads
import rioxarray
# Path to the GeoTIFF file
tif_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\PCLTILES\pcltile_75000-35000.tif"

# Directory for TIFF files
#tif_dir = r'C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\scratch\dem'

# Read the GeoTIFF file using rioxarray
extent_data = rioxarray.open_rasterio(tif_path)

# Get the spatial extent of the GeoTIFF
min_lon, min_lat, max_lon, max_lat = extent_data.rio.bounds()

graph = ox.graph_from_bbox(max_lat, min_lat, max_lon, min_lon, network_type='drive', simplify=True)

gdf = ox.graph_to_gdfs(graph, nodes=False)

# Convert the graph to GeoDataFrames
nodes, edges = ox.graph_to_gdfs(graph)

# Plot the waterway network
fig, ax = plt.subplots(figsize=(10, 10))
edges.plot(ax=ax, linewidth=1, edgecolor='blue')
#plt.title('Waterways in Fort Collins')
plt.show()


import rasterio
from rasterio.features import rasterize
from rasterio.windows import from_bounds
import numpy as np
import scipy.ndimage

# Define the decay constant
lambda_decay = 0.03  # This value might need adjustment based on your specific use case

# Load the reference TIFF to use for transformations and metadata
with rasterio.open(tif_path) as src:

    min_lon, min_lat, max_lon, max_lat = src.bounds
    
    meta = src.meta.copy()

    # Convert geographic coordinates to pixel coordinates
    window = from_bounds(min_lon, min_lat, max_lon, max_lat, src.transform)
    transform = rasterio.windows.transform(window, src.transform)

    # Correct the window dimensions to integers
    window_width = int(window.width)
    window_height = int(window.height)

    # Create an empty raster array for the size of the window
    raster = np.zeros((window_height, window_width), dtype=np.uint8)

    # Rasterize directly into the window shape
    shapes = ((geom, 1) for geom in gdf['geometry'])
    burned = rasterize(shapes, out=raster, fill=0, transform=transform, all_touched=True)

    # Compute the Euclidean distance from each cell to the nearest non-zero cell
    distance_grid = scipy.ndimage.distance_transform_edt(burned == 0)

    # Apply exponential decay to the distance grid
    decay_grid = np.exp(-lambda_decay * distance_grid)

    # Prepare metadata for saving the decay grid
    clipped_meta = meta.copy()
    clipped_meta.update({
        "driver": "GTiff",
        "height": window_height,
        "width": window_width,
        "transform": transform,
        "dtype": rasterio.float32,
        "count": 1,
        "compress": 'lzw'
    })

    # Save the decay grid to a new TIFF file
    output_path = r'C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\scratch\\roads\\exponential_decay_CO_roads.tif'
    with rasterio.open(output_path, 'w', **clipped_meta) as dst:
        dst.write(decay_grid.astype(np.float32), 1)

from osgeo import gdal, ogr, osr

# Define the paths to your rasters
#lidarHAG_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\exponential_decay_CO_river.tif"
#pcl_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\pcl_west_wgs_CO_3.tif"

# Open the source (lidarHAG) and target (pcl) datasets
src_ds = gdal.Open(output_path, gdal.GA_ReadOnly)
target_ds = gdal.Open(tif_path, gdal.GA_ReadOnly)


# Get the Geotransform and Projection from the target dataset
target_transform = target_ds.GetGeoTransform()
target_projection = target_ds.GetProjection()
target_cols = target_ds.RasterXSize
target_rows = target_ds.RasterYSize

# Create a new dataset for output with the same size and projection as the target
driver = gdal.GetDriverByName('GTiff')
output_path_resample = r"C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\scratch\\roads\\output_resampled_dataRoads.tif"
out_ds = driver.Create(output_path_resample, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
out_ds.SetGeoTransform(target_transform)
out_ds.SetProjection(target_projection)

# Perform the resampling
gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

# Close the datasets
src_ds = None
target_ds = None
out_ds = None

print("Resampling completed. Output saved at:", output_path)

# Check if the file exists before trying to delete it
if os.path.exists(output_path):
    try:
        os.remove(output_path)
        print(f"File {output_path} deleted successfully.")
    except Exception as e:
        print(f"Failed to delete {output_path}: {e}")
else:
    print(f"No file found at {output_path}, deletion not required.")


In [ ]:
### for Sentinel
import rioxarray
from pystac_client import Client
import planetary_computer
from concurrent.futures import ThreadPoolExecutor
import rioxarray
import matplotlib.pyplot as plt
import os
import glob
import subprocess
from osgeo import gdal, ogr, osr
from pystac.extensions.eo import EOExtension as eo
import pyproj
import stackstac
import numpy as np
import gc

# Path to the GeoTIFF file
tif_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\PCLTILES\pcltile_75000-40000.tif"

# Directory for TIFF files
tif_dir = r'C:\Users\smdur\OneDrive\Desktop\scratch'

# Read the GeoTIFF file using rioxarray
tif_data = rioxarray.open_rasterio(tif_path)

# Get the spatial extent of the GeoTIFF
min_lon, min_lat, max_lon, max_lat = tif_data.rio.bounds()
bbox_of_interest = [min_lon, min_lat, max_lon, max_lat] 

# Create a catalog search for the specified collection and bounding box
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

time_of_interest = "2021-07-01/2021-08-01"

search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}},
)
items = list(search.get_items())
print(f"Returned {len(items)} items")

items = search.item_collection()

base_crs = tif_data.rio.crs
base_transform = tif_data.rio.transform()
base_resolution = (tif_data.rio.width, tif_data.rio.height)
print("export")

# def process_item(item, idx):
#     signed_asset1 = planetary_computer.sign(item.assets["B08"])
#     data1 = rioxarray.open_rasterio(signed_asset1.href).squeeze().drop("band")
#     output_tif_path1 = os.path.join(tif_dir, f"SenDataBand8_data_{idx}.tif")
#     data1.rio.to_raster(output_tif_path1)

#     signed_asset2 = planetary_computer.sign(item.assets["B04"])
#     data2 = rioxarray.open_rasterio(signed_asset2.href).squeeze().drop("band")
#     output_tif_path2 = os.path.join(tif_dir, f"SenDataBand4_data_{idx}.tif")
#     data2.rio.to_raster(output_tif_path2)

import numpy as np

def process_item(item, idx):
    # Sign the assets using the planetary_computer token
    signed_asset1 = planetary_computer.sign(item.assets["B08"])
    signed_asset2 = planetary_computer.sign(item.assets["B04"])
    
    # Open the raster data and drop the band dimension
    data1 = rioxarray.open_rasterio(signed_asset1.href).squeeze().drop("band")
    data2 = rioxarray.open_rasterio(signed_asset2.href).squeeze().drop("band")
    
    output_tif_path1 = os.path.join(tif_dir, f"SSSSenDataBand8_data_{idx}.tif")
    output_tif_path2 = os.path.join(tif_dir, f"SSSSenDataBand4_data_{idx}.tif")
    
    # Check for non-zero data in the first dataset
    if np.any(data1.values != 0):
        print(f"Data for Band 8 at index {idx} contains non-zero values.")
        data1.rio.to_raster(output_tif_path1)  # Uncomment this to save when needed
    else:
        print(f"Skipping saving Band 8 data at index {idx} due to only zero values.")

    # Check for non-zero data in the second dataset
    if np.any(data2.values != 0):
        print(f"Data for Band 4 at index {idx} contains non-zero values.")
        data2.rio.to_raster(output_tif_path2)  # Uncomment this to save when needed
    else:
        print(f"Skipping saving Band 4 data at index {idx} due to only zero values.")



with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers based on your CPU
    for i, item in enumerate(items):  # Change this line to process only the last three items
        executor.submit(process_item, item, len(items) - 3 + i)  # Adjust index for correct labeling
        gc.collect()
        
print('merge')






# import os
# import glob
# import numpy as np
# from osgeo import gdal

# def read_raster(file_path):
#     return gdal.Open(file_path, gdal.GA_ReadOnly)

# def create_output_raster(template_ds, output_path):
#     driver = gdal.GetDriverByName('GTiff')
#     out_ds = driver.CreateCopy(output_path, template_ds, 0)
#     out_ds.GetRasterBand(1).Fill(0)  # Initialize with the lowest value expected
#     return out_ds

# tif_dir = "C:\\Users\\smdur\\OneDrive\\Desktop\\scratch\\"
# output_tif_path = os.path.join(tif_dir, "median_merged_0check.tif")

# # Get the list of TIFF files
# tif_files = glob.glob(os.path.join(tif_dir, "SenDataBand8_data_*.tif"))
# if not tif_files:
#     raise FileNotFoundError("No TIFF files found in the specified directory.")

# # Initialize the output raster using the first TIFF as a template
# template_ds = read_raster(tif_files[0])
# output_ds = create_output_raster(template_ds, output_tif_path)
# output_band = output_ds.GetRasterBand(1)

# # Read data from all TIFFs
# data_arrays = []
# for tif_path in tif_files:
#     ds = read_raster(tif_path)
#     if ds:
#         band = ds.GetRasterBand(1)
#         data = band.ReadAsArray()
#         data_arrays.append(data)
#     ds = None  # Ensure the file is closed after reading

# # Stack arrays and compute the median
# if data_arrays:
#     stacked_data = np.dstack(data_arrays)  # Stack along the third dimension
#     median_data = np.median(stacked_data, axis=2)  # Compute median along the third dimension

#     # Write the result back to the output raster
#     output_band.WriteArray(median_data)

# Finalize
output_band.FlushCache()
output_ds = None  # Close and save the dataset
print(f"Merged TIFF saved to {output_tif_path}")

import os
import glob
import numpy as np
from osgeo import gdal

def read_raster(file_path):
    """Opens a raster file for reading."""
    return gdal.Open(file_path, gdal.GA_ReadOnly)

def create_output_raster(template_ds, output_path):
    """Creates a new raster file based on the template with initialization."""
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.CreateCopy(output_path, template_ds, 0)
    out_ds.GetRasterBand(1).Fill(0)  # Initialize with the lowest value expected
    return out_ds

tif_dir = "C:\\Users\\smdur\\OneDrive\\Desktop\\scratch\\"
output_tif_path = os.path.join(tif_dir, "median_merged_0check.tif")

# Get the list of TIFF files
tif_files = glob.glob(os.path.join(tif_dir, "SenDataBand8_data_*.tif"))
if not tif_files:
    raise FileNotFoundError("No TIFF files found in the specified directory.")

# Initialize the output raster using the first TIFF as a template
template_ds = read_raster(tif_files[0])
output_ds = create_output_raster(template_ds, output_tif_path)
output_band = output_ds.GetRasterBand(1)

# Read data from all TIFFs and filter out all-zero files
data_arrays = []
for tif_path in tif_files:
    ds = read_raster(tif_path)
    if ds:
        band = ds.GetRasterBand(1)
        data = band.ReadAsArray()
        if np.any(data != 0):  # Check if there's any non-zero data
            data_arrays.append(data)
        else:
            print(f"Skipped {tif_path} because it contains only zeros.")
        ds = None  # Ensure the file is closed after reading

# Stack arrays and compute the median if there are valid datasets
if data_arrays:
    stacked_data = np.dstack(data_arrays)  # Stack along the third dimension
    median_data = np.median(stacked_data, axis=2)  # Compute median along the third dimension
    output_band.WriteArray(median_data)  # Write the result back to the output raster
else:
    print("No valid data found in any TIFF files.")

# Finalize
output_band.FlushCache()
output_ds = None  # Close and save the dataset
print(f"Merged TIFF saved to {output_tif_path}")


# ########################################################################################################################
# ########################################################################################################################
# ########################################################################################################################
# tif_dir = "C:\\Users\\smdur\\OneDrive\\Desktop\\scratch\\"

# output_tif1 = os.path.join(tif_dir, f"outputtile_sentinel_8_3.tif")
# print(output_tif1)
# # Get the list of TIFF files
# tifs = glob.glob(os.path.join(tif_dir, "SenDataBand8_data_*.tif"))

# # Prepare the gdal_merge command
# merge_command_hag = [
#     "python",
#     "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
#     "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
#     "-o", output_tif1,
#     #"-n", "-9999",
#     #"-a_nodata", "-9999"
#     "-n", "0",
#     "-a_nodata", "0"
# ] + tifs

# # Run the gdal_merge command and capture the output
# process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# # # Open the source (lidarHAG) and target (pcl) datasets
# src_ds = gdal.Open(output_tif1, gdal.GA_ReadOnly)
# target_ds = gdal.Open(tif_path, gdal.GA_ReadOnly)

# # Get the Geotransform and Projection from the target dataset
# target_transform = target_ds.GetGeoTransform()
# target_projection = target_ds.GetProjection()
# target_cols = target_ds.RasterXSize
# target_rows = target_ds.RasterYSize

# # Create a new dataset for output with the same size and projection as the target
# driver = gdal.GetDriverByName('GTiff')
# output_path1 = os.path.join(tif_dir, f"output_resampled_dataSEN_8.tif")

# out_ds = driver.Create(output_path1, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
# out_ds.SetGeoTransform(target_transform)
# out_ds.SetProjection(target_projection)

# # Perform the resampling
# gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

# src_ds = None
# target_ds = None
# out_ds = None

# os.remove(output_tif1)

# if process_hag.returncode != 0:
#     print(process_hag.stderr)
# else:
#     for tif in tifs:
#         try:
#             os.remove(tif)
#         except Exception as e:
#             print(f"Failed to delete {tif}: {e}")

# print("Resampling completed. Output saved at:", output_path1)

# ########################################################################################################################
# ########################################################################################################################
# ########################################################################################################################
# output_tif2 = os.path.join(tif_dir, f"outputtile_sentinel_4.tif")

# # Get the list of TIFF files
# tifs = glob.glob(os.path.join(tif_dir, "SenDataBand4_data_*.tif"))

# # Prepare the gdal_merge command
# merge_command_hag = [
#     "python",
#     "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
#     "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
#     "-o", output_tif2,
#     #"-n", "-9999",
#     #"-a_nodata", "-9999"
#     "-n", "0",
#     "-a_nodata", "0"
# ] + tifs

# # Run the gdal_merge command and capture the output
# process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# # Open the source (lidarHAG) and target (pcl) datasets
# src_ds = gdal.Open(output_tif2, gdal.GA_ReadOnly)
# target_ds = gdal.Open(tif_path, gdal.GA_ReadOnly)

# # Get the Geotransform and Projection from the target dataset
# target_transform = target_ds.GetGeoTransform()
# target_projection = target_ds.GetProjection()
# target_cols = target_ds.RasterXSize
# target_rows = target_ds.RasterYSize

# # Create a new dataset for output with the same size and projection as the target
# driver = gdal.GetDriverByName('GTiff')
# output_path2 = os.path.join(tif_dir, f"output_resampled_dataSEN_4.tif")

# out_ds = driver.Create(output_path2, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
# out_ds.SetGeoTransform(target_transform)
# out_ds.SetProjection(target_projection)

# # Perform the resampling
# gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

# src_ds = None
# target_ds = None
# out_ds = None

# os.remove(output_tif2)

# if process_hag.returncode != 0:
#     print(process_hag.stderr)
# else:
#     for tif in tifs:
#         try:
#             os.remove(tif)
#         except Exception as e:
#             print(f"Failed to delete {tif}: {e}")

# print("Resampling completed. Output saved at:", output_path2)
# ########################################################################################################################
# ########################################################################################################################
# ########################################################################################################################



In [ ]:
import rioxarray
import os

# Path to the output NDVI TIFF
output_tif3 = os.path.join(tif_dir, "outputtile_sentinel_NDVI.tif")

# Load the NIR and Red band data
nir_data = rioxarray.open_rasterio(output_path1)
red_data = rioxarray.open_rasterio(output_path2)

# Calculate NDVI
ndvi = (nir_data - red_data) / (nir_data + red_data)
ndvi = ndvi.fillna(0)  # Optional: handle NaNs by replacing them with zero

# Set nodata values if necessary
ndvi.rio.write_nodata(-9999, inplace=True)

# Save the NDVI to a new TIFF file
ndvi.rio.to_raster(output_tif3)

print(f"NDVI calculated and saved to {output_tif3}")


# Generate Random Points

In [ ]:
import rasterio
import matplotlib.pyplot as plt
from rasterio.plot import show
from shapely.geometry import box, Point
from geopandas import GeoDataFrame
import numpy as np

# Load the GeoTIFF file
file_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\PCLTILES\pcltile_75000-35000.tif"
with rasterio.open(file_path) as src:
    bounds = src.bounds
    crs = src.crs
    img = src.read(1)  # Read the first band

# Create a box from the bounds
rect = box(bounds.left, bounds.bottom, bounds.right, bounds.top)

# Apply a negative buffer to contract the boundary of the rectangle
buffered_rect = rect.buffer(-0.15)  # Adjust buffer size as needed

# Generate random points within the buffered rectangle
def generate_random_points(geometry, num_points):
    points = []
    min_x, min_y, max_x, max_y = geometry.bounds
    while len(points) < num_points:
        random_point = Point(np.random.uniform(min_x, max_x), np.random.uniform(min_y, max_y))
        if random_point.within(geometry):
            points.append(random_point)
    return points

random_points = generate_random_points(buffered_rect, 10000)

# Convert these points to a GeoDataFrame
gdf_points = GeoDataFrame(geometry=random_points, crs=crs)

# Convert points GeoDataFrame to the same CRS as the raster for accurate overlay
gdf_points = gdf_points.to_crs(crs)

# Convert geometries from the CRS to WGS84 for latitude and longitude
gdf_points_wgs84 = gdf_points.to_crs(epsg=4326)

# Extract the latitude and longitude
lat_long = gdf_points_wgs84.geometry.apply(lambda geom: (geom.y, geom.x)).tolist()

# Plotting the raster and the points
fig, ax = plt.subplots(figsize=(10, 10))
show(img, ax=ax, transform=src.transform, cmap='gray')  # Show the raster
gdf_points.plot(ax=ax, color='red', markersize=10)  # Plot the points
plt.title('Random Points on Raster')
plt.show()


# Create a training chip dataset using those random points

In [ ]:
import rasterio
from rasterio.windows import Window
import numpy as np

# Paths to the raster files
resampled_lidar_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\scratch\lidar\output_resampled_dataLIDAR.tif"
resampled_dem_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\scratch\dem\output_resampled_dataDEM.tif"
resampled_rivers_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\scratch\rivers\output_resampled_dataRivers.tif"
resampled_roads_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\scratch\roads\output_resampled_dataRoads.tif"
PCL_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\PCLTILES\pcltile_75000-35000.tif"

chip_size = 128  

for i, (lat, lon) in enumerate(lat_long):
    try:
        paths = [resampled_lidar_path, resampled_dem_path, resampled_rivers_path, resampled_roads_path, PCL_path]
        labels = ['lidar', 'dem', 'rivers', 'roads', 'pcllabels']
        
        for path, label in zip(paths, labels):
            with rasterio.open(path) as src:
                # Convert the geographic coordinates to pixel coordinates
                col, row = src.index(lon, lat)
                
                # Calculate the window
                window = Window(col - chip_size // 2, row - chip_size // 2, chip_size, chip_size)
                
                # Read the data within the window
                chip_data = src.read(1, window=window)
                
                # Prepare metadata for the chip raster
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": chip_size,
                    "width": chip_size,
                    "transform": src.window_transform(window)
                })

                # Save the chip data to a new file
                chip_output_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\inputchips\\{label}\\{label.upper()}_Chip_{i}.tif"

                # Check if the chip data is valid before saving
                if chip_data.shape == (chip_size, chip_size) and np.any(chip_data != src.nodata):
                    with rasterio.open(chip_output_path, "w", **out_meta) as dest:
                        dest.write(chip_data, 1)
                    #print(f"{label.capitalize()} chip {i} created and saved to: {chip_output_path}")
                else:
                    print(f"Skipping {label} chip {i} because it is not properly shaped or is filled with nodata.")

    except Exception as e:
        print(f"An error occurred while processing chip {i}: {e}")


# Load chips and normalize

In [ ]:
import os
import rasterio
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Paths to datasets
featurepath1 = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\inputchips\LIDAR"
featurepath2 = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\inputchips\DEM"
featurepath3 = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\inputchips\ROADS"
featurepath4 = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\inputchips\RIVERS"


labelspath = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\inputchips\pcllabels"

# Function to load GeoTIFF images as numpy arrays
def load_geotiff(path):
    with rasterio.open(path) as src:
        return src.read(1)

# Load datasets
hag_images = [load_geotiff(os.path.join(featurepath1, f)) for f in os.listdir(featurepath1) if f.endswith('.tif')]
dem_images = [load_geotiff(os.path.join(featurepath2, f)) for f in os.listdir(featurepath2) if f.endswith('.tif')]
roads_images = [load_geotiff(os.path.join(featurepath3, f)) for f in os.listdir(featurepath3) if f.endswith('.tif')]
rivers_images = [load_geotiff(os.path.join(featurepath4, f)) for f in os.listdir(featurepath4) if f.endswith('.tif')]

label_images = [load_geotiff(os.path.join(labelspath, f)) for f in os.listdir(labelspath) if f.endswith('.tif')]

# Convert lists to numpy arrays
hag_images = np.array(hag_images).astype('float32')
dem_images = np.array(dem_images).astype('float32')
roads_images = np.array(roads_images).astype('float32')
rivers_images = np.array(rivers_images).astype('float32')

label_images = np.array(label_images).astype('float32')

# Normalize images independently
hag_max = hag_images.max()
dem_max = dem_images.max()
roads_max = roads_images.max()
rivers_max = rivers_images.max()


hag_images /= hag_max
dem_images /= dem_max
roads_images /= roads_max
rivers_images /= rivers_max


In [ ]:


# Stack features along the last dimension
feature_images = np.stack((hag_images, dem_images, roads_images, rivers_images), axis=-1)

# Normalize labels if they range from 0 to 100
label_images /= 100

# Reshape labels for CNN input
label_images = np.expand_dims(label_images, axis=-1)

# Define the CNN model
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 4)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dropout(0.5),
    Dense(128 * 128, activation='sigmoid'),
    tf.keras.layers.Reshape((128, 128, 1))
])

# Define custom weights for each feature
weights = np.array([1.0, 0.8, 0.5, 0.3])  # Example weights for each feature

# Create sample weights based on the custom weights
sample_weights = np.dot(feature_images, weights)

lr = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Compile and train the model with sample weights
model.compile(optimizer=optimizer, loss='mse')
model.fit(feature_images, label_images, batch_size=128, epochs=10, validation_split=0.3, sample_weight=sample_weights)


# lr = 0.01
# optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# # Compile and train the model
# model.compile(optimizer=optimizer, loss='mse')
# model.fit(feature_images, label_images, batch_size=128, epochs=10, validation_split=0.3)


In [ ]:
print(hag_max)
print(dem_max)
print(roads_max)
print(rivers_max)

# Load or save models 

In [ ]:
#pip install --no-cache-dir --force-reinstall h5py

#model.save('C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\SavedModels\\model_4_30_LR01.h5')  # Saves the model in HDF5 format

from tensorflow.keras.models import load_model
import h5py

# Path to the model
model_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\SavedModels\model_4_30_LR01.h5"
#model_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\SavedModels\model_4_30.h5"
#model_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\SavedModels\model_4_30_LR0001.h5"
#model_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\SavedModels\model_4_29.h5"

# Load the model
model = load_model(model_path)


# Prep data for inference

In [ ]:
# this ensures all tiles are widthtile by heighttile
import os
from itertools import product
import rasterio as rio
from rasterio import windows

in_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\scratch\\'
input_filename1 = 'lidar\\output_resampled_dataLIDAR.tif'
input_filename2 = 'dem\\output_resampled_dataDEM.tif'
input_filename3 = 'roads\\output_resampled_dataRoads.tif'
input_filename4 = 'rivers\\output_resampled_dataRivers.tif'

out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\inferencetiles\\hag\\'
output_filename = 'tile_{}-{}.tif'
widthtile = 128
heighttile = 128


def get_tiles(ds, width=widthtile, height=heighttile):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    #offsets = product(range(0, nols, 1000), range(0, nrows, 1000))

    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in offsets:
        window = windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform




with rio.open(os.path.join(in_path, input_filename1)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)



out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\inferencetiles\\dem\\'
#output_filename = 'tile_{}-{}.tif'
#widthtile = 128
#heighttile = 128

with rio.open(os.path.join(in_path, input_filename2)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)

out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\inferencetiles\\roads\\'
#output_filename = 'tile_{}-{}.tif'
#widthtile = 128
#heighttile = 128

with rio.open(os.path.join(in_path, input_filename3)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)

out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\inferencetiles\\rivers\\'
#output_filename = 'tile_{}-{}.tif'
#widthtile = 128
#heighttile = 128

with rio.open(os.path.join(in_path, input_filename4)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)

print("Done")


# Make a list of tile names

In [ ]:
import os
import re

# Define the directory path
directory_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\inferencetiles\\hag'

# Regular expression to extract the identifier part of the filename 'tile_{identifier}.tif'
pattern = re.compile(r'tile_(\d+-\d+)\.tif')

# List all files in the directory
files = os.listdir(directory_path)

# Use a set to avoid duplicate identifiers
identifiers = set()

# Extract identifiers from filenames
for file in files:
    match = pattern.search(file)
    if match:
        identifiers.add(match.group(1))

# Convert the set to a sorted list
identifier_list = sorted(list(identifiers))
print(len(identifier_list))
print("done")


# Make Inference on those tiles

In [ ]:
import os
import rasterio
import numpy as np
import tensorflow as tf

#tilename = '0-0'
# input_hag_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\hag\\tile_{tilename}.tif"
# input_dem_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\dem\\tile_{tilename}.tif"
# input_roads_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\roads\\tile_{tilename}.tif"
# input_rivers_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\rivers\\tile_{tilename}.tif"

def load_and_preprocess_image(hag_path, dem_path, roads_path, rivers_path):
    with rasterio.open(hag_path) as src:
        hag_image = src.read(1)
    with rasterio.open(dem_path) as src:
        dem_image = src.read(1)
    with rasterio.open(roads_path) as src:
        roads_image = src.read(1)
    with rasterio.open(rivers_path) as src:
        rivers_image = src.read(1)

    # Normalize and stack the images
    hag_image = np.array(hag_image).astype('float32') / hag_max
    dem_image = np.array(dem_image).astype('float32') / dem_max
    roads_image = np.array(roads_image).astype('float32') / roads_max
    rivers_image = np.array(rivers_image).astype('float32') / rivers_max

    # Stack images along the last dimension
    combined_image = np.stack([hag_image, dem_image, roads_image, rivers_image], axis=-1)

    # Add batch dimension
    combined_image = np.expand_dims(combined_image, axis=0)
    return combined_image



for i in range(len(identifier_list)):
    tilename = identifier_list[i]
    #print(tilename)
    input_hag_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\inferencetiles\\hag\\tile_{tilename}.tif"
    input_dem_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\inferencetiles\\dem\\tile_{tilename}.tif"
    input_roads_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\inferencetiles\\roads\\tile_{tilename}.tif"
    input_rivers_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\inferencetiles\\rivers\\tile_{tilename}.tif"


    input_image = load_and_preprocess_image(input_hag_path, input_dem_path, input_roads_path, input_rivers_path)
    predicted_image = model.predict(input_image)
    predicted_image = np.squeeze(predicted_image)
    
    # Debug print to check if all outputs are the same
    #print("Unique values in predicted output:", np.unique(predicted_image))
    
    # Adjust the scaling factor based on how the labels were scaled during training
    predicted_image *= 100
    
    output_image_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\PCLCONUS\\Input\\predictions\\predicted_tile_{tilename}.tif"
    
    with rasterio.open(input_dem_path) as src: 
        profile = src.profile
    
    with rasterio.open(output_image_path, 'w', **profile) as dst:
        dst.write(predicted_image.astype(rasterio.uint8), 1)

print("Done")


# Merge files 

In [ ]:
import os
import glob
import subprocess

# Input and output directories
tif_dir = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\predictions"
output_dir = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\merged"
output_base_name = "predMerged_CO_"  # Base name for output files

# Get a list of TIFF files
tifs = glob.glob(os.path.join(tif_dir, "*.tif"))

# Define chunk size for processing
chunk_size = 300

# Calculate the number of chunks needed
num_chunks = len(tifs) // chunk_size
if len(tifs) % chunk_size != 0:
    num_chunks += 1  # Add one more chunk for the remaining files

# Loop through the TIFF files in chunks
for chunk_id in range(num_chunks):
    # Calculate the start and end indices for the current chunk
    start_idx = chunk_id * chunk_size
    end_idx = min((chunk_id + 1) * chunk_size, len(tifs))

    # Get the current chunk of TIFF files
    chunk_tifs = tifs[start_idx:end_idx]
    
    # Prepare the output file name
    output_tif = os.path.join(output_dir, f"{output_base_name}{chunk_id + 1}.tif")

    # Prepare the gdal_merge command for the current chunk
    merge_command_hag = [
        "python",
        "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
        "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
        #"-ot", "Byte",
        "-o", output_tif,
        "-n", "-9999",
        "-a_nodata", "-9999",
    ] + chunk_tifs

    # Run the gdal_merge command for the current chunk
    process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Check if the command for the current chunk was successful
    if process_hag.returncode != 0:
        # An error occurred, print the error
        print(f"Error occurred while merging TIFF files for chunk {chunk_id + 1}:")
        print(process_hag.stderr)
    else:
        print(f"TIFF files merged successfully for chunk {chunk_id + 1}. Output: {output_tif}")



# Merge the Merged

In [ ]:
### WORKS WITH NO BOARDER!
import os
import glob
import subprocess

#tif_dir = r"D:\GlobalPCL\lidarHAG"
#output_tif = r"D:\GlobalPCL\lidarHAG\LIDAR_GEDI_CO.tif"
tif_dir = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\merged"
output_tif  = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\merged\predMerged_PCL_london.tif"


tifs = glob.glob(os.path.join(tif_dir, "*.tif"))
print(len(tifs))

# Prepare the gdal_merge command for HAG
merge_command_hag = [
    "python",
    "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
    "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
    #"-ot", "Byte",
    "-o", output_tif,
    "-n", "-9999",
    "-a_nodata","-9999",    
] + tifs

# Run the gdal_merge command for HAG and capture the output
process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Check if the command for HAG was successful
if process_hag.returncode != 0:
    # An error occurred, print the error
    print("Error occurred while merging TIFF files HAG:")
    print(process_hag.stderr)
else:
    print("TIFF files merged successfully for HAG.")

for tif in tifs:
    try:
        os.remove(tif)
        print(f"Deleted {tif}")
    except Exception as e:
        print(f"Failed to delete {tif}: {e}")


# Plot

In [ ]:
import os
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

# Path to the first TIFF file
#tif_path1 = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\merged\predMerged_PCL.tif"
tif_path1 = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\predMerged_PCL_lr0001.tif"
# Path to the second TIFF file
tif_path2 = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\PCLTILES\pcltile_75000-35000.tif"

# Open the first TIFF file
with rasterio.open(tif_path1) as src1:
    # Read the raster data
    raster_data1 = src1.read(1)
    # Get the colormap for the first raster
    cmap1 = plt.cm.viridis
    
# Open the second TIFF file
with rasterio.open(tif_path2) as src2:
    # Read the raster data
    raster_data2 = src2.read(1)
    # Get the colormap for the second raster
    cmap2 = plt.cm.viridis
    
# Create a figure with subplots for the first raster
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))  # Reduced height to 8 inches

# Plot the histogram for the first raster
axes[0, 0].hist(raster_data1.flatten(), bins=60, color='black', alpha=0.7)
axes[0, 0].set_xlabel('Pixel Values')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Histogram of predicted PCL Values')
axes[0, 0].grid(True)

# Plot the first raster image
img1 = axes[0, 1].imshow(raster_data1, cmap=cmap1, norm=Normalize(vmin=raster_data1.min(), vmax=raster_data1.max()))
#axes[0, 1].set_title('Raster 1 Image')
axes[0, 1].set_xlabel('X')
axes[0, 1].set_ylabel('Y')
fig.colorbar(img1, ax=axes[0, 1], label='Pixel Value')

# Plot the histogram for the second raster
axes[1, 0].hist(raster_data2.flatten(), bins=60, color='black', alpha=0.7)
axes[1, 0].set_xlabel('Pixel Values')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Histogram of actual PCL Values')
axes[1, 0].grid(True)

# Plot the second raster image
img2 = axes[1, 1].imshow(raster_data2, cmap=cmap2, norm=Normalize(vmin=raster_data1.min(), vmax=raster_data1.max()))
#axes[1, 1].set_title('Raster 2 Image')
axes[1, 1].set_xlabel('X')
axes[1, 1].set_ylabel('Y')
fig.colorbar(img2, ax=axes[1, 1], label='Pixel Value')

# Adjust layout and show the plot
plt.tight_layout()

plt.savefig(r"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\gifs_figs\\modelpred1LR0001.png")

plt.show()


# GLOBAL INFRARENCE

In [ ]:
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import transform
import numpy as np
import rioxarray
from pystac_client import Client
import planetary_computer
from concurrent.futures import ThreadPoolExecutor
import os
import glob
import subprocess
from osgeo import gdal

def create_tile(center_lat, center_lon, reference_raster_path, output_tile_path):
    # Open the reference raster to get its properties
    with rasterio.open(reference_raster_path) as ref_raster:
        ref_transform = ref_raster.transform
        ref_crs = ref_raster.crs
        ref_res = ref_transform[0]
        ref_width = ref_raster.width
        ref_height = ref_raster.height
        new_profile = ref_raster.profile.copy()
    
    # Convert lat/lon to the same coordinate system as the reference raster
    dst_crs = ref_crs
    src_crs = 'EPSG:4326'
    center_x, center_y = transform(src_crs, dst_crs, [center_lon], [center_lat])
    center_x, center_y = center_x[0], center_y[0]
    
    # Define the new transform for the 5000x5000 tile
    tile_width = 5000
    tile_height = 5000
    tile_transform = from_origin(
        center_x - (tile_width // 2) * ref_res,
        center_y + (tile_height // 2) * ref_res,
        ref_res,
        ref_res
    )
    
    # Create the new raster data (a tile filled with ones)
    tile_data = np.ones((tile_height, tile_width), dtype=np.uint8)
    
    # Define the new raster profile
    new_profile.update({
        'height': tile_height,
        'width': tile_width,
        'transform': tile_transform,
        'dtype': 'uint8'
    })
    
    # Write the new tile to a file
    with rasterio.open(output_tile_path, 'w', **new_profile) as dst_raster:
        dst_raster.write(tile_data, 1)

# Define the center latitude and longitude
#center_lat = 40.7128  # Example: New York City latitude
#center_lon = -74.0060  # Example: New York City longitude

# Define the center latitude and longitude
center_lat = 51.5074  # Example: London
center_lon = -0.1278  # Example: UK

# Define the reference raster path and output tile path
reference_raster_path = r"C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\PCLTILES\pcltile_75000-40000.tif"
output_tile_path = r"C:\Users\smdur\OneDrive\Desktop\scratch2\outputs\tileLondon.tif"

# Create the tile
create_tile(center_lat, center_lon, reference_raster_path, output_tile_path)

In [ ]:
### for DEM_GLOBAL
import rioxarray
from pystac_client import Client
import planetary_computer
from concurrent.futures import ThreadPoolExecutor
import rioxarray
import matplotlib.pyplot as plt
import os
import glob
import subprocess
from osgeo import gdal, ogr, osr

# Path to the GeoTIFF file
tif_path = output_tile_path

# Directory for TIFF files
tif_dir = r'C:\Users\smdur\OneDrive\Desktop\scratch2\dem'

# Read the GeoTIFF file using rioxarray
tif_data = rioxarray.open_rasterio(tif_path)

# Get the spatial extent of the GeoTIFF
min_lon, min_lat, max_lon, max_lat = tif_data.rio.bounds()
bbox_of_interest = [min_lon, min_lat, max_lon, max_lat]  # Replace with your actual bounding box coordinates

# Create a catalog search for the specified collection and bounding box
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search = catalog.search(collections=["cop-dem-glo-30"], bbox=bbox_of_interest)
items = list(search.get_items())
print(f"Returned {len(items)} items")

def process_item(item, idx):
    #print(f"Processing item {idx+1}/{len(items)}")
    signed_asset = planetary_computer.sign(item.assets["data"])
    data = (
        rioxarray.open_rasterio(signed_asset.href)
        .squeeze()
        .drop("band")
    )
    data.rio.write_crs("EPSG:4326", inplace=True)

    output_tif_path = os.path.join(tif_dir, f"output_dataDEM_{idx}.tif")
    data.rio.to_raster(output_tif_path)

with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers based on your CPU
    for i, item in enumerate(items):  # Change this line to process only the last three items
        executor.submit(process_item, item, len(items) - 3 + i)  # Adjust index for correct labeling

output_tif = os.path.join(tif_dir, f"outputtile_DEM.tif")

# Get the list of TIFF files
tifs = glob.glob(os.path.join(tif_dir, "*.tif"))

# Prepare the gdal_merge command
merge_command_hag = [
    "python",
    "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
    "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
    "-o", output_tif,
    "-n", "-9999",
    "-a_nodata", "-9999"
] + tifs

# Run the gdal_merge command and capture the output
process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Open the source (lidarHAG) and target (pcl) datasets
src_ds = gdal.Open(output_tif, gdal.GA_ReadOnly)
target_ds = gdal.Open(tif_path, gdal.GA_ReadOnly)

# Get the Geotransform and Projection from the target dataset
target_transform = target_ds.GetGeoTransform()
target_projection = target_ds.GetProjection()
target_cols = target_ds.RasterXSize
target_rows = target_ds.RasterYSize

# Create a new dataset for output with the same size and projection as the target
driver = gdal.GetDriverByName('GTiff')
output_path = os.path.join(tif_dir, f"output_resampled_dataDEMlondon.tif")

out_ds = driver.Create(output_path, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
out_ds.SetGeoTransform(target_transform)
out_ds.SetProjection(target_projection)

# Perform the resampling
gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

src_ds = None
target_ds = None
out_ds = None


if process_hag.returncode != 0:
    print(process_hag.stderr)
else:
    for tif in tifs:
        try:
            os.remove(tif)
        except Exception as e:
            print(f"Failed to delete {tif}: {e}")

print("Resampling completed. Output saved at:", output_path)

os.remove(output_tif)


In [ ]:
### for LIDAR_GLOBAL
import os
import glob
import subprocess
from osgeo import gdal, ogr, osr

tif_dir = r"C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\lidar2"

tifs = glob.glob(os.path.join(tif_dir, "*.tif"))

output_tif = os.path.join(tif_dir, f"outputtile_lidar.tif")

# Get the list of TIFF files
tifs = glob.glob(os.path.join(tif_dir, "*.tif"))

# Prepare the gdal_merge command
merge_command_hag = [
    "python",
    "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
    "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
    "-o", output_tif,
    "-n", "255",
    "-a_nodata", "255"
] + tifs

# Run the gdal_merge command and capture the output
process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Open the source (lidarHAG) and target (pcl) datasets
src_ds = gdal.Open(output_tif, gdal.GA_ReadOnly)
target_ds = gdal.Open(tif_path, gdal.GA_ReadOnly)

# Get the Geotransform and Projection from the target dataset
target_transform = target_ds.GetGeoTransform()
target_projection = target_ds.GetProjection()
target_cols = target_ds.RasterXSize
target_rows = target_ds.RasterYSize

# Create a new dataset for output with the same size and projection as the target
driver = gdal.GetDriverByName('GTiff')
#output_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\CNNPCLDEMO\Input\Resampled_DEM_CO.tif"
output_path = os.path.join(tif_dir, f"output_resampled_dataLIDARlondon.tif")

out_ds = driver.Create(output_path, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
out_ds.SetGeoTransform(target_transform)
out_ds.SetProjection(target_projection)

# Perform the resampling
gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

# Close the datasets
src_ds = None
target_ds = None
out_ds = None

os.remove(output_tif)

print("Resampling completed. Output saved at:", output_path)

In [ ]:
# for Rivers
import rioxarray
import osmnx as ox

# Path to the GeoTIFF file
#tif_path = r"C:\Users\smdur\OneDrive\Desktop\scratch2\outputs\tile3london.tif"
tif_path = output_tile_path
# Directory for TIFF files
#tif_dir = r'C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\scratch\dem'

# Read the GeoTIFF file using rioxarray
dem_data = rioxarray.open_rasterio(tif_path)

# Get the spatial extent of the GeoTIFF
min_lon, min_lat, max_lon, max_lat = dem_data.rio.bounds()

#custom_filter = '["waterway"~"river|streams|canals"]'
custom_filter = '["waterway"~"river"]'

bbox = (max_lat, min_lat, max_lon, min_lon)  
graph = ox.graph_from_bbox(bbox=bbox, custom_filter=custom_filter, simplify=True, retain_all=True, truncate_by_edge=True)


gdf = ox.graph_to_gdfs(graph, nodes=False)

# Convert the graph to GeoDataFrames
nodes, edges = ox.graph_to_gdfs(graph)

import rasterio
from rasterio.features import rasterize
from rasterio.windows import from_bounds
import numpy as np
import scipy.ndimage

# Define the decay constant
lambda_decay = 0.03  # This value might need adjustment based on your specific use case

# Load the reference TIFF to use for transformations and metadata
with rasterio.open(tif_path) as src:

    min_lon, min_lat, max_lon, max_lat = src.bounds
    
    meta = src.meta.copy()

    # Convert geographic coordinates to pixel coordinates
    window = from_bounds(min_lon, min_lat, max_lon, max_lat, src.transform)
    transform = rasterio.windows.transform(window, src.transform)

    # Correct the window dimensions to integers
    window_width = int(window.width)
    window_height = int(window.height)

    # Create an empty raster array for the size of the window
    raster = np.zeros((window_height, window_width), dtype=np.uint8)

    # Rasterize directly into the window shape
    shapes = ((geom, 1) for geom in gdf['geometry'])
    burned = rasterize(shapes, out=raster, fill=0, transform=transform, all_touched=True)

    # Compute the Euclidean distance from each cell to the nearest non-zero cell
    distance_grid = scipy.ndimage.distance_transform_edt(burned == 0)

    # Apply exponential decay to the distance grid
    decay_grid = np.exp(-lambda_decay * distance_grid)

    # Prepare metadata for saving the decay grid
    clipped_meta = meta.copy()
    clipped_meta.update({
        "driver": "GTiff",
        "height": window_height,
        "width": window_width,
        "transform": transform,
        "dtype": rasterio.float32,
        "count": 1,
        "compress": 'lzw'
    })

    # Save the decay grid to a new TIFF file
    output_path = r'C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\rivers\\exponential_decay_CO_river.tif'
    with rasterio.open(output_path, 'w', **clipped_meta) as dst:
        dst.write(decay_grid.astype(np.float32), 1)

from osgeo import gdal, ogr, osr

# Define the paths to your rasters
#lidarHAG_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\exponential_decay_CO_river.tif"
#pcl_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\Input\pcl_west_wgs_CO_3.tif"

# Open the source (lidarHAG) and target (pcl) datasets
src_ds = gdal.Open(output_path, gdal.GA_ReadOnly)
target_ds = gdal.Open(tif_path, gdal.GA_ReadOnly)


# Get the Geotransform and Projection from the target dataset
target_transform = target_ds.GetGeoTransform()
target_projection = target_ds.GetProjection()
target_cols = target_ds.RasterXSize
target_rows = target_ds.RasterYSize

# Create a new dataset for output with the same size and projection as the target
driver = gdal.GetDriverByName('GTiff')
output_path_resample = r"C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\rivers\\output_resampled_dataRiverslondon.tif"
out_ds = driver.Create(output_path_resample, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
out_ds.SetGeoTransform(target_transform)
out_ds.SetProjection(target_projection)

# Perform the resampling
gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

# Close the datasets
src_ds = None
target_ds = None
out_ds = None

print("Resampling completed. Output saved at:", output_path)

# Check if the file exists before trying to delete it
if os.path.exists(output_path):
    try:
        os.remove(output_path)
        print(f"File {output_path} deleted successfully.")
    except Exception as e:
        print(f"Failed to delete {output_path}: {e}")
else:
    print(f"No file found at {output_path}, deletion not required.")


In [ ]:
# for Roads_GLOBAL
import rioxarray
# Path to the GeoTIFF file
#tif_path = r"C:\Users\smdur\OneDrive\Desktop\scratch2\outputs\tile3.tif"
tif_path = output_tile_path
# Directory for TIFF files
#tif_dir = r'C:\Users\smdur\OneDrive\Desktop\PCLCONUS\Input\scratch\dem'

# Read the GeoTIFF file using rioxarray
extent_data = rioxarray.open_rasterio(tif_path)

# Get the spatial extent of the GeoTIFF
min_lon, min_lat, max_lon, max_lat = extent_data.rio.bounds()

graph = ox.graph_from_bbox(max_lat, min_lat, max_lon, min_lon, network_type='drive', simplify=True)

gdf = ox.graph_to_gdfs(graph, nodes=False)

# Convert the graph to GeoDataFrames
nodes, edges = ox.graph_to_gdfs(graph)

# Plot the waterway network
fig, ax = plt.subplots(figsize=(10, 10))
edges.plot(ax=ax, linewidth=1, edgecolor='blue')
#plt.title('Waterways in Fort Collins')
plt.show()


import rasterio
from rasterio.features import rasterize
from rasterio.windows import from_bounds
import numpy as np
import scipy.ndimage

# Define the decay constant
lambda_decay = 0.03  # This value might need adjustment based on your specific use case

# Load the reference TIFF to use for transformations and metadata
with rasterio.open(tif_path) as src:

    min_lon, min_lat, max_lon, max_lat = src.bounds
    
    meta = src.meta.copy()

    # Convert geographic coordinates to pixel coordinates
    window = from_bounds(min_lon, min_lat, max_lon, max_lat, src.transform)
    transform = rasterio.windows.transform(window, src.transform)

    # Correct the window dimensions to integers
    window_width = int(window.width)
    window_height = int(window.height)

    # Create an empty raster array for the size of the window
    raster = np.zeros((window_height, window_width), dtype=np.uint8)

    # Rasterize directly into the window shape
    shapes = ((geom, 1) for geom in gdf['geometry'])
    burned = rasterize(shapes, out=raster, fill=0, transform=transform, all_touched=True)

    # Compute the Euclidean distance from each cell to the nearest non-zero cell
    distance_grid = scipy.ndimage.distance_transform_edt(burned == 0)

    # Apply exponential decay to the distance grid
    decay_grid = np.exp(-lambda_decay * distance_grid)

    # Prepare metadata for saving the decay grid
    clipped_meta = meta.copy()
    clipped_meta.update({
        "driver": "GTiff",
        "height": window_height,
        "width": window_width,
        "transform": transform,
        "dtype": rasterio.float32,
        "count": 1,
        "compress": 'lzw'
    })

    # Save the decay grid to a new TIFF file
    output_path = r'C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\roads\\exponential_decay_CO_roads.tif'
    with rasterio.open(output_path, 'w', **clipped_meta) as dst:
        dst.write(decay_grid.astype(np.float32), 1)

from osgeo import gdal, ogr, osr

# Open the source (lidarHAG) and target (pcl) datasets
src_ds = gdal.Open(output_path, gdal.GA_ReadOnly)
target_ds = gdal.Open(tif_path, gdal.GA_ReadOnly)


# Get the Geotransform and Projection from the target dataset
target_transform = target_ds.GetGeoTransform()
target_projection = target_ds.GetProjection()
target_cols = target_ds.RasterXSize
target_rows = target_ds.RasterYSize

# Create a new dataset for output with the same size and projection as the target
driver = gdal.GetDriverByName('GTiff')
output_path_resample = r"C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\roads\\output_resampled_dataRoadslondon.tif"
out_ds = driver.Create(output_path_resample, target_cols, target_rows, 1, src_ds.GetRasterBand(1).DataType)
out_ds.SetGeoTransform(target_transform)
out_ds.SetProjection(target_projection)

# Perform the resampling
gdal.ReprojectImage(src_ds, out_ds, src_ds.GetProjection(), target_projection, gdal.GRA_Bilinear)

# Close the datasets
src_ds = None
target_ds = None
out_ds = None

print("Resampling completed. Output saved at:", output_path)

# Check if the file exists before trying to delete it
if os.path.exists(output_path):
    try:
        os.remove(output_path)
        print(f"File {output_path} deleted successfully.")
    except Exception as e:
        print(f"Failed to delete {output_path}: {e}")
else:
    print(f"No file found at {output_path}, deletion not required.")


In [ ]:
# this ensures all tiles are widthtile by heighttile
import os
from itertools import product
import rasterio as rio
from rasterio import windows

in_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\'
input_filename1 = 'lidar\\output_resampled_dataLIDARlondon.tif'
input_filename2 = 'dem\\output_resampled_dataDEMlondon.tif'
input_filename3 = 'roads\\output_resampled_dataRoads.tif'
input_filename4 = 'rivers\\output_resampled_dataRivers.tif'

out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\outputinferencetiles\\hag2\\'
output_filename = 'tile_{}-{}.tif'
widthtile = 128
heighttile = 128


def get_tiles(ds, width=widthtile, height=heighttile):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    #offsets = product(range(0, nols, 1000), range(0, nrows, 1000))

    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in offsets:
        window = windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform




with rio.open(os.path.join(in_path, input_filename1)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)



out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\outputinferencetiles\\dem2\\'
#output_filename = 'tile_{}-{}.tif'
#widthtile = 128
#heighttile = 128

with rio.open(os.path.join(in_path, input_filename2)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)

out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\outputinferencetiles\\roads2\\'
#output_filename = 'tile_{}-{}.tif'
#widthtile = 128
#heighttile = 128

with rio.open(os.path.join(in_path, input_filename3)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)

out_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\outputinferencetiles\\rivers2\\'
#output_filename = 'tile_{}-{}.tif'
#widthtile = 128
#heighttile = 128

with rio.open(os.path.join(in_path, input_filename4)) as inds:
    tile_width, tile_height = widthtile, heighttile
    nodata = inds.nodata  # Get the NoData value from the dataset
    meta = inds.meta.copy()

    for window, transform in get_tiles(inds):
        if window.width == tile_width and window.height == tile_height:  # Check if the tile dimensions are as expected
            data = inds.read(window=window)
            if nodata is not None:
                # Modified check for NoData to include tolerance for floating-point rasters
                valid_data_mask = (data != nodata)
            else:
                # If NoData value is not set, consider all data as valid
                valid_data_mask = (data == data)

            if valid_data_mask.any():  # Check if there's any valid data within the tile
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path, output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(data)

print("Done")


In [ ]:
import os
import re

# Define the directory path
directory_path = 'C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\outputinferencetiles\\hag2'

# Regular expression to extract the identifier part of the filename 'tile_{identifier}.tif'
pattern = re.compile(r'tile_(\d+-\d+)\.tif')

# List all files in the directory
files = os.listdir(directory_path)

# Use a set to avoid duplicate identifiers
identifiers = set()

# Extract identifiers from filenames
for file in files:
    match = pattern.search(file)
    if match:
        identifiers.add(match.group(1))

# Convert the set to a sorted list
identifier_list = sorted(list(identifiers))
print(len(identifier_list))
print("done")


In [ ]:
print("dd")

In [ ]:
from tensorflow.keras.models import load_model
import h5py

# Path to the model
#model_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\SavedModels\model_4_30_LR01.h5"
#model_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\SavedModels\model_4_30.h5"
#model_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\SavedModels\model_4_30_LR0001.h5"
#model_path = r"C:\Users\smdur\OneDrive\Desktop\GLOBALPCL\SavedModels\model_4_29.h5"

#model_path = r"D:\SandraComputer\GLOBALPCL\SavedModels\model_4_30.h5"
#model_path = r"D:\SandraComputer\GLOBALPCL\SavedModels\model_4_30_LR0001.h5"
model_path = r"D:\SandraComputer\GLOBALPCL\SavedModels\model_4_30_LR01.h5"


# Load the model
model = load_model(model_path)

In [ ]:
import os
import rasterio
import numpy as np
import tensorflow as tf

#tilename = '0-0'
# input_hag_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\hag\\tile_{tilename}.tif"
# input_dem_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\dem\\tile_{tilename}.tif"
# input_roads_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\roads\\tile_{tilename}.tif"
# input_rivers_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\GLOBALPCL\\CNNPCLDEMO\\inferencetiles\\rivers\\tile_{tilename}.tif"

def load_and_preprocess_image(hag_path, dem_path, roads_path, rivers_path):
    with rasterio.open(hag_path) as src:
        hag_image = src.read(1)
    with rasterio.open(dem_path) as src:
        dem_image = src.read(1)
    with rasterio.open(roads_path) as src:
        roads_image = src.read(1)
    with rasterio.open(rivers_path) as src:
        rivers_image = src.read(1)

    # Normalize and stack the images
    hag_image = np.array(hag_image).astype('float32') / hag_max
    dem_image = np.array(dem_image).astype('float32') / dem_max
    roads_image = np.array(roads_image).astype('float32') / roads_max
    rivers_image = np.array(rivers_image).astype('float32') / rivers_max

    # Stack images along the last dimension
    combined_image = np.stack([hag_image, dem_image, roads_image, rivers_image], axis=-1)

    # Add batch dimension
    combined_image = np.expand_dims(combined_image, axis=0)
    return combined_image



for i in range(len(identifier_list)):
    tilename = identifier_list[i]
    #print(tilename)
    input_hag_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\outputinferencetiles\\hag2\\tile_{tilename}.tif"
    input_dem_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\outputinferencetiles\\dem2\\tile_{tilename}.tif"
    input_roads_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\outputinferencetiles\\roads2\\tile_{tilename}.tif"
    input_rivers_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\outputinferencetiles\\rivers2\\tile_{tilename}.tif"


    input_image = load_and_preprocess_image(input_hag_path, input_dem_path, input_roads_path, input_rivers_path)
    predicted_image = model.predict(input_image)
    predicted_image = np.squeeze(predicted_image)
    
    # Debug print to check if all outputs are the same
    #print("Unique values in predicted output:", np.unique(predicted_image))
    
    # Adjust the scaling factor based on how the labels were scaled during training
    predicted_image *= 100
    
    output_image_path = f"C:\\Users\\smdur\\OneDrive\\Desktop\\scratch2\\predictions\\predicted_tile_{tilename}.tif"
    
    with rasterio.open(input_dem_path) as src: 
        profile = src.profile
    
    with rasterio.open(output_image_path, 'w', **profile) as dst:
        dst.write(predicted_image.astype(rasterio.uint8), 1)

print("Done")


In [ ]:
import os
import glob
import subprocess

# Input and output directories
tif_dir = r"C:\Users\smdur\OneDrive\Desktop\scratch2\predictions"
output_dir = r"C:\Users\smdur\OneDrive\Desktop\scratch2\outputpred"
output_base_name = "predMerged_CO_"  # Base name for output files

# Get a list of TIFF files
tifs = glob.glob(os.path.join(tif_dir, "*.tif"))

# Define chunk size for processing
chunk_size = 300

# Calculate the number of chunks needed
num_chunks = len(tifs) // chunk_size
if len(tifs) % chunk_size != 0:
    num_chunks += 1  # Add one more chunk for the remaining files

# Loop through the TIFF files in chunks
for chunk_id in range(num_chunks):
    # Calculate the start and end indices for the current chunk
    start_idx = chunk_id * chunk_size
    end_idx = min((chunk_id + 1) * chunk_size, len(tifs))

    # Get the current chunk of TIFF files
    chunk_tifs = tifs[start_idx:end_idx]
    
    # Prepare the output file name
    output_tif = os.path.join(output_dir, f"{output_base_name}{chunk_id + 1}.tif")

    # Prepare the gdal_merge command for the current chunk
    merge_command_hag = [
        "python",
        "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
        "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
        #"-ot", "Byte",
        "-o", output_tif,
        "-n", "-9999",
        "-a_nodata", "-9999",
    ] + chunk_tifs

    # Run the gdal_merge command for the current chunk
    process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Check if the command for the current chunk was successful
    if process_hag.returncode != 0:
        # An error occurred, print the error
        print(f"Error occurred while merging TIFF files for chunk {chunk_id + 1}:")
        print(process_hag.stderr)
    else:
        print(f"TIFF files merged successfully for chunk {chunk_id + 1}. Output: {output_tif}")



In [ ]:
### WORKS WITH NO BOARDER!
import os
import glob
import subprocess

#tif_dir = r"D:\GlobalPCL\lidarHAG"
#output_tif = r"D:\GlobalPCL\lidarHAG\LIDAR_GEDI_CO.tif"
tif_dir = r"C:\Users\smdur\OneDrive\Desktop\scratch2\outputpred"
output_tif  = r"C:\Users\smdur\OneDrive\Desktop\scratch2\outputs\predMerged_PCL_5_13_m1_london.tif"


tifs = glob.glob(os.path.join(tif_dir, "*.tif"))
print(len(tifs))

# Prepare the gdal_merge command for HAG
merge_command_hag = [
    "python",
    "C:\\Users\\smdur\\anaconda3\\envs\\globalpcl\\Scripts\\gdal_merge.py",
    "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
    #"-ot", "Byte",
    "-o", output_tif,
    "-n", "-9999",
    "-a_nodata","-9999",    
] + tifs

# Run the gdal_merge command for HAG and capture the output
process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Check if the command for HAG was successful
if process_hag.returncode != 0:
    # An error occurred, print the error
    print("Error occurred while merging TIFF files HAG:")
    print(process_hag.stderr)
else:
    print("TIFF files merged successfully for HAG.")

for tif in tifs:
    try:
        os.remove(tif)
        print(f"Deleted {tif}")
    except Exception as e:
        print(f"Failed to delete {tif}: {e}")
